In [1]:
import sib_ldsc_cython as ld

In [4]:
import numpy as np
import glob
import pandas as pd
from scipy.optimize import minimize
import scipy.optimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats

In [7]:
V = np.array([[0.5, 0.25], [0.25, 0.5]])
ld.calc_inv_root(V)

array([[ 1.57735027, -0.42264973],
       [-0.42264973,  1.57735027]])